The result from the previous notebook:

- LGBM RMSLE Score --> 0.5461

Another Experiment for LGBM:

- Without outlier mapper --> 0.5452
- Without missing value Imputation --> 0.5461
- Without feature restriction --> 0.5888
- Add pickdrop cluster --> 0.5556
- Add skewness transform --> 0.5480
- Add dummy encoding --> 0.5549

## Stage Selection

In [1]:
from utils_ml import *

In [2]:
df = pd.read_csv("csv_ml/eda_01.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 14 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   id                       1458644 non-null  object 
 1   vendor_id                1458644 non-null  int64  
 2   pickup_datetime          1458644 non-null  object 
 3   dropoff_datetime         1458644 non-null  object 
 4   passenger_count          1458644 non-null  int64  
 5   pickup_longitude         1458644 non-null  float64
 6   pickup_latitude          1458644 non-null  float64
 7   dropoff_longitude        1458644 non-null  float64
 8   dropoff_latitude         1458644 non-null  float64
 9   store_and_fwd_flag       1458644 non-null  object 
 10  trip_duration            1458644 non-null  int64  
 11  distance_osrm            1458627 non-null  float64
 12  pickup_dist_NYC_center   1458644 non-null  float64
 13  dropoff_dist_NYC_center  1458644 non-null 

In [4]:
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,distance_osrm,pickup_dist_NYC_center,dropoff_dist_NYC_center
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,2.1587,6.445619,6.825864
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,2.5113,3.585708,2.111028
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,9.9354,6.118901,0.306521
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,1.7776,0.866388,0.858280
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,1.6064,9.352795,8.230915


In [5]:
X = df.drop(columns="trip_duration", axis=1)
y = df["trip_duration"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [7]:
%%time
rm_duplicates = RemoveDuplicates()
X_train, y_train = rm_duplicates.transform(X_train, y_train)

>>>> Starting the process of removing duplicates ...
No duplicates found.
CPU times: total: 5.92 s
Wall time: 6.96 s


In [8]:
%%time
to_dtypes = ToDataTypes()
X_train, y_train = to_dtypes.transform(X_train, y_train)
X_test, y_test = to_dtypes.transform(X_test, y_test)

>>>> Starting data type conversion process...
Column 'vendor_id' changed from int64 to object
Column 'pickup_datetime' changed from object to datetime64[ns]
Column 'dropoff_datetime' changed from object to datetime64[ns]
>>>> Starting data type conversion process...
Column 'vendor_id' changed from int64 to object
Column 'pickup_datetime' changed from object to datetime64[ns]
Column 'dropoff_datetime' changed from object to datetime64[ns]
CPU times: total: 5.27 s
Wall time: 5.32 s


In [9]:
%%time
datetime_break = DateTimeBreak()
X_train = datetime_break.transform(X_train)
X_test = datetime_break.transform(X_test)

>>>> Starting datetime feature extraction...
Extracted features: ['pickup_month', 'pickup_day', 'pickup_day_of_week', 'pickup_hour']
Dropped columns: ['pickup_datetime', 'dropoff_datetime']
>>>> Starting datetime feature extraction...
Extracted features: ['pickup_month', 'pickup_day', 'pickup_day_of_week', 'pickup_hour']
Dropped columns: ['pickup_datetime', 'dropoff_datetime']
CPU times: total: 5.89 s
Wall time: 6.37 s


In [10]:
%time
miss_val_input = MissValInput()
X_train, y_train = miss_val_input.fit_transform(X_train, y_train)
X_test, y_test = miss_val_input.transform(X_test, y_test)

CPU times: total: 0 ns
Wall time: 0 ns
>>>> Starting missing value imputation...
Dropped 16 entries (0.00%) from column 'distance_osrm' due to missing values.
Initial data length: 1166915
Removed data: 16 (0.00%)
Final data length: 1166899
>>>> Starting missing value imputation...
Dropped 1 entries (0.00%) from column 'distance_osrm' due to missing values.
Initial data length: 291729
Removed data: 1 (0.00%)
Final data length: 291728


In [11]:
%%time
feature_restriction = FeatureRestriction()
X_train, y_train = feature_restriction.transform(X_train, y_train)

>>>> Starting features restriction ...
The dataset size: 1166899 rows
trip_duration (old) -> [min, max]: [1, 3526282]
trip_duration (new) -> [min, max]: [60, 86392]
distance_osrm (old) -> [min, max]: [0.0, 765.6445]
distance_osrm (new) -> [min, max]: [0.1001, 97.7243]
speed_osrm column not found, skipping restriction on 'speed_osrm'.
passenger_count (old) -> [min, max]: [0, 8]
passenger_count (new) -> [min, max]: [1, 8]
Total removed data: 11632 (1.00%)
CPU times: total: 1.88 s
Wall time: 1.99 s


In [12]:
%%time
outlier_mapper = OutlierMapper(map_title="outliers_map_ml_baseline", csv_dir="csv_ml_baseline", html_dir="html_ml_baseline")
X_train, y_train = outlier_mapper.transform(X_train, y_train)
X_test.drop(columns=['pickup_dist_NYC_center', 'dropoff_dist_NYC_center'], inplace=True)

>>>> Starting New York City map restriction ...
Outliers saved to 'csv_ml_baseline\outliers_map_ml_baseline.csv'
Map saved as 'html_ml_baseline/outliers_map_ml_baseline.html'
Removed 71 (0.01%) records outside NYC boundaries.
CPU times: total: 2.41 s
Wall time: 2.46 s


In [13]:
%%time
feature_encoding = FeatureEncoding()
X_train = feature_encoding.transform(X_train)
X_test = feature_encoding.transform(X_test)

>>>> Starting to encode the features ...
Starting transformations...
Dropping 'id' column...
Performing dummy encoding on 'vendor_id' column...
Performing dummy encoding on 'store_and_fwd_flag' column...
Performing cyclical encoding on 'pickup_month' column...
Cyclical encoding for 'pickup_month' completed.
Performing cyclical encoding on 'pickup_day' column...
Cyclical encoding for 'pickup_day' completed.
Performing cyclical encoding on 'pickup_day_of_week' column...
Cyclical encoding for 'pickup_day_of_week' completed.
Performing cyclical encoding on 'pickup_hour' column...
Cyclical encoding for 'pickup_hour' completed.
Dropping original columns after encoding...
Transformation completed.
>>>> Starting to encode the features ...
Starting transformations...
Dropping 'id' column...
Performing dummy encoding on 'vendor_id' column...
Performing dummy encoding on 'store_and_fwd_flag' column...
Performing cyclical encoding on 'pickup_month' column...
Cyclical encoding for 'pickup_month' co

In [14]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1155196 entries, 1053743 to 121958
Data columns (total 16 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   passenger_count         1155196 non-null  int64  
 1   pickup_longitude        1155196 non-null  float64
 2   pickup_latitude         1155196 non-null  float64
 3   dropoff_longitude       1155196 non-null  float64
 4   dropoff_latitude        1155196 non-null  float64
 5   distance_osrm           1155196 non-null  float64
 6   vendor_id_2             1155196 non-null  uint8  
 7   store_and_fwd_flag_Y    1155196 non-null  uint8  
 8   pickup_month_sin        1155196 non-null  float64
 9   pickup_month_cos        1155196 non-null  float64
 10  pickup_day_sin          1155196 non-null  float64
 11  pickup_day_cos          1155196 non-null  float64
 12  pickup_day_of_week_sin  1155196 non-null  float64
 13  pickup_day_of_week_cos  1155196 non-null  float64
 1

In [15]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 291728 entries, 67250 to 589044
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   passenger_count         291728 non-null  int64  
 1   pickup_longitude        291728 non-null  float64
 2   pickup_latitude         291728 non-null  float64
 3   dropoff_longitude       291728 non-null  float64
 4   dropoff_latitude        291728 non-null  float64
 5   distance_osrm           291728 non-null  float64
 6   vendor_id_2             291728 non-null  uint8  
 7   store_and_fwd_flag_Y    291728 non-null  uint8  
 8   pickup_month_sin        291728 non-null  float64
 9   pickup_month_cos        291728 non-null  float64
 10  pickup_day_sin          291728 non-null  float64
 11  pickup_day_cos          291728 non-null  float64
 12  pickup_day_of_week_sin  291728 non-null  float64
 13  pickup_day_of_week_cos  291728 non-null  float64
 14  pickup_hour_sin 

In [16]:
models = [
    ('LGBM', LGBMRegressor(verbose=0))
]

In [17]:
result = evaluate_models(models, X_train, y_train, X_test, y_test)

LGBM - RMSLE: 0.5461, Fit time: 14.8824 seconds


## GridSearchCV

In [1]:
from utils_ml import *

In [2]:
df = pd.read_csv("csv_ml/eda_01.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 14 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   id                       1458644 non-null  object 
 1   vendor_id                1458644 non-null  int64  
 2   pickup_datetime          1458644 non-null  object 
 3   dropoff_datetime         1458644 non-null  object 
 4   passenger_count          1458644 non-null  int64  
 5   pickup_longitude         1458644 non-null  float64
 6   pickup_latitude          1458644 non-null  float64
 7   dropoff_longitude        1458644 non-null  float64
 8   dropoff_latitude         1458644 non-null  float64
 9   store_and_fwd_flag       1458644 non-null  object 
 10  trip_duration            1458644 non-null  int64  
 11  distance_osrm            1458627 non-null  float64
 12  pickup_dist_NYC_center   1458644 non-null  float64
 13  dropoff_dist_NYC_center  1458644 non-null 

In [4]:
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,distance_osrm,pickup_dist_NYC_center,dropoff_dist_NYC_center
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,2.1587,6.445619,6.825864
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,2.5113,3.585708,2.111028
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,9.9354,6.118901,0.306521
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,1.7776,0.866388,0.858280
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,1.6064,9.352795,8.230915


In [5]:
X = df.drop(columns="trip_duration", axis=1)
y = df["trip_duration"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [7]:
%%time
rm_duplicates = RemoveDuplicates()
X_train, y_train = rm_duplicates.transform(X_train, y_train)

>>>> Starting the process of removing duplicates ...
No duplicates found.
CPU times: total: 5.64 s
Wall time: 5.91 s


In [8]:
%%time
to_dtypes = ToDataTypes()
X_train, y_train = to_dtypes.transform(X_train, y_train)
X_test, y_test = to_dtypes.transform(X_test, y_test)

>>>> Starting data type conversion process...
Column 'vendor_id' changed from int64 to object
Column 'pickup_datetime' changed from object to datetime64[ns]
Column 'dropoff_datetime' changed from object to datetime64[ns]
>>>> Starting data type conversion process...
Column 'vendor_id' changed from int64 to object
Column 'pickup_datetime' changed from object to datetime64[ns]
Column 'dropoff_datetime' changed from object to datetime64[ns]
CPU times: total: 5.31 s
Wall time: 5.37 s


In [9]:
%%time
datetime_break = DateTimeBreak()
X_train = datetime_break.transform(X_train)
X_test = datetime_break.transform(X_test)

>>>> Starting datetime feature extraction...
Extracted features: ['pickup_month', 'pickup_day', 'pickup_day_of_week', 'pickup_hour']
Dropped columns: ['pickup_datetime', 'dropoff_datetime']
>>>> Starting datetime feature extraction...
Extracted features: ['pickup_month', 'pickup_day', 'pickup_day_of_week', 'pickup_hour']
Dropped columns: ['pickup_datetime', 'dropoff_datetime']
CPU times: total: 5.84 s
Wall time: 6.13 s


In [10]:
%time
miss_val_input = MissValInput()
X_train, y_train = miss_val_input.fit_transform(X_train, y_train)
X_test, y_test = miss_val_input.transform(X_test, y_test)

CPU times: total: 0 ns
Wall time: 0 ns
>>>> Starting missing value imputation...
Dropped 16 entries (0.00%) from column 'distance_osrm' due to missing values.
Initial data length: 1166915
Removed data: 16 (0.00%)
Final data length: 1166899
>>>> Starting missing value imputation...
Dropped 1 entries (0.00%) from column 'distance_osrm' due to missing values.
Initial data length: 291729
Removed data: 1 (0.00%)
Final data length: 291728


In [11]:
%%time
feature_restriction = FeatureRestriction()
X_train, y_train = feature_restriction.transform(X_train, y_train)

>>>> Starting features restriction ...
The dataset size: 1166899 rows
trip_duration (old) -> [min, max]: [1, 3526282]
trip_duration (new) -> [min, max]: [60, 86392]
distance_osrm (old) -> [min, max]: [0.0, 765.6445]
distance_osrm (new) -> [min, max]: [0.1001, 97.7243]
speed_osrm column not found, skipping restriction on 'speed_osrm'.
passenger_count (old) -> [min, max]: [0, 8]
passenger_count (new) -> [min, max]: [1, 8]
Total removed data: 11632 (1.00%)
CPU times: total: 1.88 s
Wall time: 2.03 s


In [12]:
%%time
outlier_mapper = OutlierMapper(map_title="outliers_map_ml_baseline", csv_dir="csv_ml_baseline", html_dir="html_ml_baseline")
X_train, y_train = outlier_mapper.transform(X_train, y_train)
X_test.drop(columns=['pickup_dist_NYC_center', 'dropoff_dist_NYC_center'], inplace=True)

>>>> Starting New York City map restriction ...
Outliers saved to 'csv_ml_baseline\outliers_map_ml_baseline.csv'
Map saved as 'html_ml_baseline/outliers_map_ml_baseline.html'
Removed 71 (0.01%) records outside NYC boundaries.
CPU times: total: 2.58 s
Wall time: 2.6 s


In [13]:
%%time
feature_encoding = FeatureEncoding()
X_train = feature_encoding.transform(X_train)
X_test = feature_encoding.transform(X_test)

>>>> Starting to encode the features ...
Starting transformations...
Dropping 'id' column...
Performing dummy encoding on 'vendor_id' column...
Performing dummy encoding on 'store_and_fwd_flag' column...
Performing cyclical encoding on 'pickup_month' column...
Cyclical encoding for 'pickup_month' completed.
Performing cyclical encoding on 'pickup_day' column...
Cyclical encoding for 'pickup_day' completed.
Performing cyclical encoding on 'pickup_day_of_week' column...
Cyclical encoding for 'pickup_day_of_week' completed.
Performing cyclical encoding on 'pickup_hour' column...
Cyclical encoding for 'pickup_hour' completed.
Dropping original columns after encoding...
Transformation completed.
>>>> Starting to encode the features ...
Starting transformations...
Dropping 'id' column...
Performing dummy encoding on 'vendor_id' column...
Performing dummy encoding on 'store_and_fwd_flag' column...
Performing cyclical encoding on 'pickup_month' column...
Cyclical encoding for 'pickup_month' co

In [14]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1155196 entries, 1053743 to 121958
Data columns (total 16 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   passenger_count         1155196 non-null  int64  
 1   pickup_longitude        1155196 non-null  float64
 2   pickup_latitude         1155196 non-null  float64
 3   dropoff_longitude       1155196 non-null  float64
 4   dropoff_latitude        1155196 non-null  float64
 5   distance_osrm           1155196 non-null  float64
 6   vendor_id_2             1155196 non-null  uint8  
 7   store_and_fwd_flag_Y    1155196 non-null  uint8  
 8   pickup_month_sin        1155196 non-null  float64
 9   pickup_month_cos        1155196 non-null  float64
 10  pickup_day_sin          1155196 non-null  float64
 11  pickup_day_cos          1155196 non-null  float64
 12  pickup_day_of_week_sin  1155196 non-null  float64
 13  pickup_day_of_week_cos  1155196 non-null  float64
 1

In [15]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 291728 entries, 67250 to 589044
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   passenger_count         291728 non-null  int64  
 1   pickup_longitude        291728 non-null  float64
 2   pickup_latitude         291728 non-null  float64
 3   dropoff_longitude       291728 non-null  float64
 4   dropoff_latitude        291728 non-null  float64
 5   distance_osrm           291728 non-null  float64
 6   vendor_id_2             291728 non-null  uint8  
 7   store_and_fwd_flag_Y    291728 non-null  uint8  
 8   pickup_month_sin        291728 non-null  float64
 9   pickup_month_cos        291728 non-null  float64
 10  pickup_day_sin          291728 non-null  float64
 11  pickup_day_cos          291728 non-null  float64
 12  pickup_day_of_week_sin  291728 non-null  float64
 13  pickup_day_of_week_cos  291728 non-null  float64
 14  pickup_hour_sin 

In [16]:
model = LGBMRegressor(random_state=42, verbose=-1)

In [17]:
# result = evaluate_models(models, X_train, y_train, X_test, y_test)

In [18]:
# Create a custom scorer for GridSearchCV
rmsle_scorer = make_scorer(rmsle, greater_is_better=False)

In [19]:
param_grid = {
    'n_estimators': [100, 200],   # Number of boosting rounds
    'learning_rate': [0.05, 0.1],     # Step size shrinkage
    'num_leaves': [31, 63],            # Maximum leaves in a tree
    'max_depth': [10, -1]
}

In [20]:
cv = KFold(n_splits=3, random_state=42, shuffle=True)

In [21]:
# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=rmsle_scorer,
    cv=cv,
    verbose=2,
    n_jobs=-1
)

Total fit = number of fold (cv) * the number of different hyperparameter configurations

In [22]:
%%time
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
CPU times: total: 49.1 s
Wall time: 9min 15s


GridSearchCV(cv=KFold(n_splits=3, random_state=42, shuffle=True),
             estimator=LGBMRegressor(random_state=42, verbose=-1), n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.1], 'max_depth': [10, -1],
                         'n_estimators': [100, 200], 'num_leaves': [31, 63]},
             scoring=make_scorer(rmsle, greater_is_better=False), verbose=2)

In [23]:
# Best parameters and best model
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation RSMLE Score:", grid_search.best_score_)

Best Parameters: {'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 200, 'num_leaves': 31}
Best Cross-Validation RSMLE Score: -0.4892630110267507


In [24]:
print("Best: %f using %s" % (grid_search.best_score_, grid_search.best_params_))
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"{mean:.6f} (±{stdev:.6f}) with: {param}")

Best: -0.489263 using {'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 200, 'num_leaves': 31}
-0.500057 (±0.000551) with: {'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 100, 'num_leaves': 31}
-0.496747 (±0.000701) with: {'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 100, 'num_leaves': 63}
-0.489263 (±0.000216) with: {'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 200, 'num_leaves': 31}
-0.490794 (±0.001658) with: {'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 200, 'num_leaves': 63}
-0.501088 (±0.000938) with: {'learning_rate': 0.05, 'max_depth': -1, 'n_estimators': 100, 'num_leaves': 31}
-0.498772 (±0.000665) with: {'learning_rate': 0.05, 'max_depth': -1, 'n_estimators': 100, 'num_leaves': 63}
-0.489979 (±0.000566) with: {'learning_rate': 0.05, 'max_depth': -1, 'n_estimators': 200, 'num_leaves': 31}
-0.492227 (±0.000365) with: {'learning_rate': 0.05, 'max_depth': -1, 'n_estimators': 200, 'num_leaves': 63}
-0.489339 (±0.001412) with: {'lear

In [25]:
# Evaluation on test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

test_rmsle = rmsle(y_test, y_pred)
print("Test RSMLE:", test_rmsle)

Test RSMLE: 0.5462083977336132


---

In [26]:
model = [
    ('LGBM', LGBMRegressor(random_state=42, verbose=-1, n_estimators=200, learning_rate=0.05, num_leaves=31, max_depth=10))
]

In [27]:
result = evaluate_models(model, X_train, y_train, X_test, y_test)

LGBM - RMSLE: 0.5462, Fit time: 20.9654 seconds


---

In [28]:
joblib.dump(best_model, 'lgbm_model.joblib')
print("Model saved to lgbm_model.joblib")

Model saved to lgbm_model.joblib


In [29]:
loaded_model = joblib.load('lgbm_model.joblib')
print("Model loaded successfully")

Model loaded successfully


In [30]:
# Make predictions with the loaded model
y_pred = loaded_model.predict(X_test)

# Evaluate the loaded model
test_rmsle = rmsle(y_test, y_pred)
print(f"Test RSMLE with loaded model: {test_rmsle:.6f}")

Test RSMLE with loaded model: 0.546208
